In [25]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold


In [27]:
def get_Data(csv, iteration = False):
    merged_dataset = pd.read_csv(csv)

    if(iteration):
        # merged_dataset = merged_dataset[merged_dataset["Iteration"] == 0]
        # merged_dataset = merged_dataset.drop(columns=["Type"])
        grouped_data = merged_dataset.groupby(['NaCl_ppm','MgSO4_ppm', "Iteration"])

    else:
        grouped_data = merged_dataset.groupby(['NaCl_ppm','MgSO4_ppm'])

    # Create a dictionary to store grouped data
    grouped_dict = {}
    # Iterate over groups and populate the dictionary
    for group_name, group_data in grouped_data:
        # print(group_name)
        tempDict = {}
        tempDF = group_data.drop(columns=['Iteration','Time','NaCl_ppm','MgSO4_ppm'])
        for col in tempDF.columns:
            vectorizedCol = tempDF[col].values
            tempDict[col] = vectorizedCol.copy()
        grouped_dict[group_name] = tempDict

    return grouped_dict

In [28]:
# Flattens the data
def arrangeData(dict):
    compArr = []
    for vals in list(dict.values()):
        arr = []
        for val in list(vals.values()):
            arr.extend(val.copy() - np.array(val).mean())
        compArr.append(arr.copy())
    
    return compArr


In [29]:
path = '../experiment_logs_UPDT/'
file = 'merged_experiments_fixed_iter_removed_q.csv'
file_path = path + file
grouped_dict = get_Data(file_path, iteration = True)



In [30]:
columns = pd.read_csv(file_path).columns

In [31]:
# Flattening the input features to shape (num_samples, 20 * 201)
# num_samples = len(grouped_dict)
# print(num_samples)
X_flattened = arrangeData(grouped_dict)  # shape will be (num_samples, 20*201)
print(len(X_flattened))
print(len(X_flattened[0]))

6354
4020


In [32]:
X = X_flattened
# X = np.array([[val for arr in list(vals.values()) for val in arr] ])

# Adjust the labels y to include both nacl_ppm and mgso4_ppm
y = [k[:2] for k in grouped_dict.keys()]



In [10]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
def init_model(input_size, output_size, learning_rate=0.001):
    # Define the model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(201 * input_size,)),  # First hidden layer
        Dropout(0.2),  # Dropout layer for regularization
        Dense(256, activation='relu'),  # Second hidden layer
        Dropout(0.2),  # Dropout layer for regularization
        Dense(32, activation='relu'),  # Third hidden layer
        Dense(output_size, activation='linear')  # Output layer for regression (2 outputs)
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])

    # Summary of the model
    # model.summary()

    return model

In [10]:
print("X_train shape:", np.array(X_train).shape)
print("y_train shape:", np.array(y_train).shape)

X_train shape: (5083, 4020)
y_train shape: (5083, 2)


In [34]:
def get_score(model, y_true, x_test):
    y_test_pred = model.predict(np.array(x_test))
    y_pred_nacl = np.array([k[0] for k in y_test_pred])
    y_pred_mgso4 = np.array([k[1] for k in y_test_pred])
    y_test_nacl = np.array([k[0] for k in y_true])
    y_test_mgso4 = np.array([k[1] for k in y_true])
    r2_salt = r2_score(np.array(y_test_nacl), np.array(y_pred_nacl))
    r2_mg = r2_score(np.array(y_test_mgso4), np.array(y_pred_mgso4))
    mae_salt = np.mean(np.abs(np.array(y_test_nacl) - np.array(y_pred_nacl)))
    mae_mg = np.mean(np.abs(np.array(y_test_mgso4) - np.array(y_pred_mgso4)))
    return r2_salt, r2_mg, mae_salt, mae_mg


In [35]:
np.random.seed(0)
def kfold_train(X, y, input_dim, output_dim, n_splits=5, epochs=400, batch_size=32):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=np.random.randint(0, 100))
    models = []
    scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = init_model(input_dim, output_dim)
        model.fit(np.array(X_train), np.array(y_train), epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

        loss, mae = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
        r2_salt, r2_mg, mae_salt, mae_mg = get_score(model, y_test, X_test)
        scores.append({'loss': loss, 'mae': mae, 'r2_salt': r2_salt, 'r2_mg': r2_mg, 'mae_salt': mae_salt, 'mae_mg': mae_mg})
        models.append(model)

    scores = pd.DataFrame(scores)

    return models, scores

In [36]:
model, scores = kfold_train(np.array(X), np.array(y), 20, 2, n_splits=5, epochs=400, batch_size=32)
scores.to_csv('mlp_all.csv')

c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [54]:
scores.mean()

loss        8.409779e+11
mae         3.248995e+04
r2_salt    -1.471522e+04
r2_mg      -3.390335e+04
mae_salt    2.796066e+04
mae_mg      3.701924e+04
dtype: float64

In [37]:
def arrangeDataSingleColumn(dict,columns):
    compArr = []
    for vals in list(dict.values()):
        arr = []
        for col in columns:
            arr.extend(vals[col].copy() - np.array(vals[col]).mean())
        compArr.append(arr.copy())
    
    return compArr

In [58]:
np.random.seed(0)
# Impedance, ImpedancePhase, Resistance, Conductance
selected_column_X = arrangeDataSingleColumn(grouped_dict,["Impedance", "ImpedancePhase","Resistance", "Conductance"])
model_imp, scores_imp = kfold_train(np.array(selected_column_X), np.array(y), 4, 2, n_splits=5, epochs=400, batch_size=32)
scores_imp.to_csv('mlp_imp.csv')

c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [62]:
from pandas import read_csv


scores_imp = read_csv('mlp_imp_2.csv')

scores_imp.mean()

In [45]:
np.random.seed(0)
# "Ireal", "Iimag","Vreal", "Vimag"
selected_column_X = arrangeDataSingleColumn(grouped_dict,["Ireal", "Iimag","Vreal", "Vimag"])
model_v, scores_v = kfold_train(np.array(selected_column_X), np.array(y), 4, 2, n_splits=5, epochs=400, batch_size=32)
scores_v.to_csv('mlp_v.csv')

c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [51]:
print(scores_v.mean())

loss        4.315847e+06
mae         8.633972e+02
r2_salt     9.715102e-01
r2_mg       7.802472e-01
mae_salt    4.508146e+02
mae_mg      1.275980e+03
dtype: float64


In [17]:
#singular columns
resultDict = {}

for i, column in enumerate(columns):
    print(f"Processing {column} -- {i}/{len(columns)}                    ", end = '\r')
    if column == 'NaCl_ppm' or column == 'MgSO4_ppm' or column == 'Iteration' or column == 'Time':
        continue
    model = init_model(1, 2)
    selected_column_X = arrangeDataSingleColumn(grouped_dict,["Impedance"])
    X_train, X_test, y_train, y_test = train_test_split(selected_column_X, y, test_size=0.2, random_state=42)

    # Fit the model
    model.fit(np.array(X_train), np.array(y_train), epochs=400, batch_size=32, validation_split=0.2)

    # Evaluate the model on test data
    # Assuming X_test and y_test are your test datasets
    loss, mae = model.evaluate(np.array(X_test), np.array(y_test))

    r2_scre = get_score(model, y_test, X_test)

    # print(f"Test Mean Squared Error: {loss}")
    # print(f"Test Mean Absolute Error: {mae}")
    # print(f"R² score on the test set: {r2_scre}")

    resultDict[column] = {'mae' : mae, 'mse': loss, "r2": r2_scre}
    
print(f"-- Process Complete --                                             ", end = '\r')
    

c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1230713782272.0000 - mae: 26559.0098 - val_loss: 213851488.0000 - val_mae: 8553.3008
Epoch 2/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 146160762880.0000 - mae: 11130.5430 - val_loss: 124116744.0000 - val_mae: 7997.9805
Epoch 3/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 42396377088.0000 - mae: 9969.2305 - val_loss: 168203008.0000 - val_mae: 7138.1401
Epoch 4/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 56871542784.0000 - mae: 9352.6064 - val_loss: 51188560.0000 - val_mae: 5929.8945
Epoch 5/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 409289792.0000 - mae: 5911.7114 - val_loss: 38184220.0000 - val_mae: 5005.6816
Epoch 6/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 669312576.0000 - mae: 5258.1782 - val_loss: 28798708.0000 - val_mae: 4410.4668
Epoch 7/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 594656384.0000 - mae: 4969.8164 - val_loss: 25212768.0000 - val_mae: 4094.7136
Epoc

c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 324038393856.0000 - mae: 12807.1553 - val_loss: 142820864.0000 - val_mae: 8315.8486
Epoch 2/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 340239319040.0000 - mae: 13273.7656 - val_loss: 112723200.0000 - val_mae: 7426.9126
Epoch 3/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 443726757888.0000 - mae: 15365.9883 - val_loss: 64052520.0000 - val_mae: 6136.9526
Epoch 4/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 97691254784.0000 - mae: 8925.8242 - val_loss: 34544672.0000 - val_mae: 4968.4009
Epoch 5/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 180094240.0000 - mae: 5230.5195 - val_loss: 26732434.0000 - val_mae: 4355.5205
Epoch 6/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 167213296.0000 - mae: 4617.4556 - val_loss: 23574460.0000 - val_mae: 3985.0469
Epoch 7/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 149040416.0000 - mae: 4534.3203 - val_loss: 23002860.0000 - val_mae: 3837.3589
Epoc

c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Epoch 1/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 960863928320.0000 - mae: 22979.0957 - val_loss: 247057088.0000 - val_mae: 8545.8223
Epoch 2/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3976253407232.0000 - mae: 73329.6172 - val_loss: 199439104.0000 - val_mae: 8179.3032
Epoch 3/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 705616084992.0000 - mae: 23298.0430 - val_loss: 129601960.0000 - val_mae: 7658.7788
Epoch 4/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 47428489216.0000 - mae: 9375.6475 - val_loss: 92644832.0000 - val_mae: 6860.5708
Epoch 5/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19749883904.0000 - mae: 8049.4067 - val_loss: 69718832.0000 - val_mae: 6018.5669
Epoch 6/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 458040640.0000 - mae: 6062.3701 - val_loss: 56617336.0000 - val_mae: 5391.5933
Epoch 7/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 320422912.0000 - mae: 5478.2925 - val_loss: 49518340.0000 - val_mae: 5062.8818


c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 302698463232.0000 - mae: 17559.4160 - val_loss: 154226736.0000 - val_mae: 9064.3398
Epoch 2/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 38347681792.0000 - mae: 12949.7598 - val_loss: 149299440.0000 - val_mae: 8862.0654
Epoch 3/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19148083200.0000 - mae: 10858.2363 - val_loss: 144597856.0000 - val_mae: 8632.5068
Epoch 4/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9566990336.0000 - mae: 9660.0264 - val_loss: 139075360.0000 - val_mae: 8430.6924
Epoch 5/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 638871616.0000 - mae: 8460.8721 - val_loss: 166617152.0000 - val_mae: 8197.6006
Epoch 6/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 460077824.0000 - mae: 8200.4863 - val_loss: 162250384.0000 - val_mae: 8006.2183
Epoch 7/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 145648752.0000 - mae: 7872.3042 - val_loss: 166681328.0000 - val_mae: 7695.4717
Ep

c:\Users\UYSM3\miniconda3\envs\cortex\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 67313410048.0000 - mae: 11627.6826 - val_loss: 197156560.0000 - val_mae: 8783.6943
Epoch 2/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 278444507136.0000 - mae: 14947.2129 - val_loss: 180114144.0000 - val_mae: 8537.5771
Epoch 3/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 29342507008.0000 - mae: 9144.3408 - val_loss: 176521360.0000 - val_mae: 8018.5278
Epoch 4/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 389888640.0000 - mae: 7958.8311 - val_loss: 171143872.0000 - val_mae: 7489.0771
Epoch 5/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 73293056.0000 - mae: 7180.3286 - val_loss: 150128736.0000 - val_mae: 6807.4448
Epoch 6/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 109824288.0000 - mae: 6674.0591 - val_loss: 122857496.0000 - val_mae: 6078.8022
Epoch 7/400
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1072596608.0000 - mae: 6242.2617 - val_loss: 77944456.0000 - val_mae: 5275.5317
Epoch

In [47]:
model_v[0].summary()

Model: "sequential_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_152 (Dense)               │ (None, 128)            │       103,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_76 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_153 (Dense)               │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_77 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_154 (Dense)               │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_155 (Dense)               │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 433,064 (1.65 MB)

 Trainable params: 144,354 (563.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 288,710 (1.10 MB)